In [9]:
!wget https://storage.yandexcloud.net/ds-ods/files/materials/9606875c/train.csv

--2023-05-01 19:03:39--  https://storage.yandexcloud.net/ds-ods/files/materials/9606875c/train.csv
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1156382 (1.1M) [text/csv]
Saving to: ‘train.csv’

train.csv           100%[===================>]   1.10M  1.02MB/s    in 1.1s    

2023-05-01 19:03:41 (1.02 MB/s) - ‘train.csv’ saved [1156382/1156382]



In [52]:
from catboost import CatBoostClassifier,Pool,cv
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [53]:
def clear_usrer_df(df,labels):
  labels = labels.value_counts().index[:35]
  drop_index = []
  for i,j in zip(df.index,df.user_id):
    if j not in labels:
      drop_index += [i]
  return df.drop(drop_index,axis=0)

In [54]:
def date(string):
    lst = string.split(" ")
    return lst[0]

def time(string):
    lst = string.split(" ")
    return lst[1]
def preprocess_test(data):
    data["date"] = data["timestamp"].apply(date)
    data["time"] = data["timestamp"].apply(time)
    data["timestamp"] = pd.to_datetime(data["timestamp"])
    data["1day"] = data["timestamp"].dt.is_month_start.apply(lambda x: 1 if x == True else 0)
    data["last_day"] = data["timestamp"].dt.is_month_end.apply(lambda x: 1 if x == True else 0)
    data["DofW"] = data["timestamp"].dt.dayofweek
    data["DofY1"] = (data["timestamp"].dt.dayofyear % 4).apply(lambda x: 1 if x in (1, 2) else 0)
    # data["DofY2"] = (data["timestamp"].dt.dayofyear % 6).apply(lambda x: 1 if x in (0, 1, 2) else 0)
    data["month"] = data["timestamp"].dt.month
    data["day"] = data["timestamp"].dt.day
    data["is_weekend"] = data["timestamp"].apply(lambda x: 1 if x.date().weekday() in (5, 6) else 0)
    data.loc[data["month"].eq(1) & data["day"].eq(16), "is_weekend"] = 1  #16.01
    data.loc[data["month"].eq(2) & data["day"].ge(4), "is_weekend"] = 1  #after 02.03
    data.loc[data["month"].eq(9) & data["day"].eq(12), "is_weekend"] = 1  # 12.09
    data.loc[data["month"].eq(11) & data["day"].eq(11), "is_weekend"] = 1  #11.11
    data.loc[data["month"].eq(11) & data["day"].eq(16), "is_weekend"] = 1  #16.11
    data["hour"] = data["timestamp"].dt.hour
    data["m1"] = data["hour"].apply(lambda x: 1 if x == 7 else 0)
    data["m2"] = data["hour"].apply(lambda x: 1 if x == 8 else 0)
    data["m3"] = data["hour"].apply(lambda x: 1 if x == 9 else 0)
    data["m4"] = data["hour"].apply(lambda x: 1 if x == 10 else 0)
    data["d1"] = data["hour"].apply(lambda x: 1 if x == 11 else 0)
    data["d2"] = data["hour"].apply(lambda x: 1 if x == 12 else 0)
    data["d3"] = data["hour"].apply(lambda x: 1 if x == 14 else 0)
    data["d4"] = data["hour"].apply(lambda x: 1 if x == 15 else 0)
    data["e1"] = data["hour"].apply(lambda x: 1 if x == 18 else 0)
    data["e2"] = data["hour"].apply(lambda x: 1 if x == 19 else 0)
    data["e3"] = data["hour"].apply(lambda x: 1 if x == 20 else 0)
    data["e4"] = data["hour"].apply(lambda x: 1 if x == 21 else 0)
    data["e5"] = data["hour"].apply(lambda x: 1 if x == 22 else 0)
    data["e6"] = data["hour"].apply(lambda x: 1 if x == 23 else 0)
    data["min"] = data["timestamp"].dt.minute
    data["sec"] = data["timestamp"].dt.second
    data["shift"] = data["timestamp"].shift(1)
    data["shift"] = pd.to_datetime(data["shift"])
    data["lag"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s")
    data["l3s"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s")
    data["lag1"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s")
    data["lag2"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s")
    data["lag3"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s")
    data["lag4"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s")
    data["lag5"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s")
    data["lag6"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s")
    data["lag7"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s")
    data["lag8"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s")
    data["lag9"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s")
    data["lag0"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s")
    data["lagx"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s")
    data["gs1"] = data["gate_id"].shift(1)
    data["gs2"] = data["gate_id"].shift(2)
    data["gs3"] = data["gate_id"].shift(3)
    data["gs4"] = data["gate_id"].shift(4)
    data["gs5"] = data["gate_id"].shift(5)
    data["gs-1"] = data["gate_id"].shift(-1)
    data["gs-2"] = data["gate_id"].shift(-2)
    data["gs-3"] = data["gate_id"].shift(-3)
    data["gs-4"] = data["gate_id"].shift(-4)
    data["gs-5"] = data["gate_id"].shift(-5)
    data["lag"] = data["lag"].apply(lambda x: 1 if x < 2 else 0)
    data["l3s"] = data["l3s"].apply(lambda x: 1 if x <= 3 else 0)
    data["l-1"] = data["l3s"].shift(-1)
    data.loc[data["l3s"].eq(1) | data["l-1"].eq(1), "ls1"] = 1
    data["lag1"] = data["lag1"].apply(lambda x: 1 if 6 > x > 2 else 0)
    data["lag2"] = data["lag2"].apply(lambda x: 1 if 15 > x >= 6 else 0)
    data["lag3"] = data["lag3"].apply(lambda x: 1 if 22 > x >= 15 else 0)
    data["lag4"] = data["lag4"].apply(lambda x: 1 if 32 > x >= 22 else 0)
    data["lag5"] = data["lag5"].apply(lambda x: 1 if 42 > x > 32 else 0)
    data["lag6"] = data["lag6"].apply(lambda x: 1 if 58 > x >= 42 else 0)
    data["lag7"] = data["lag7"].apply(lambda x: 1 if 69 > x >= 58 else 0)
    data["lag8"] = data["lag8"].apply(lambda x: 1 if 76 > x >= 69 else 0)
    data["lag9"] = data["lag9"].apply(lambda x: 1 if 130 >= x > 127 else 0)
    data["lag0"] = data["lag0"].apply(lambda x: 0 if x > 69 else x)
    data["lagx"] = data["lagx"].apply(lambda x: 1 if x > 130 else 0)
    data["double"] = data["gate_id"][data["gate_id"].eq(data["gs1"]) & data["ls1"].eq(1) | data["gate_id"].eq(data["gs-1"])].apply(lambda x: 1 if x >= 1 else 0)
    data["double2"] = data["gate_id"][data["gate_id"].eq(data["gs1"]) & data["lag"].eq(0) | data["gate_id"].eq(data["gs-1"])].apply(lambda x: 1 if x >= 1 else 0)
    data["double3"] = data["gate_id"][data["gate_id"].eq(data["gs1"]) & data["lag1"].eq(1) | data["gate_id"].eq(data["gs-1"])].apply(lambda x: 1 if x >= 1 else 0)
    data["double4"] = data["gate_id"][data["gate_id"].eq(data["gs1"]) & data["lag2"].eq(1) | data["gate_id"].eq(data["gs-1"])].apply(lambda x: 1 if x >= 1 else 0)
    data["double5"] = data["gate_id"][data["gate_id"].eq(data["gs1"]) & data["lag3"].eq(1) | data["gate_id"].eq(data["gs-1"])].apply(lambda x: 1 if x >= 1 else 0)
    data["double6"] = data["gate_id"][data["gate_id"].eq(data["gs1"]) & data["lag4"].eq(1) | data["gate_id"].eq(data["gs-1"])].apply(lambda x: 1 if x >= 1 else 0)
    data["double7"] = data["gate_id"][data["gate_id"].eq(data["gs1"]) & data["lag7"].eq(1) | data["gate_id"].eq(data["gs-1"])].apply(lambda x: 1 if x >= 1 else 0)
    data["double8"] = data["gate_id"][data["gate_id"].eq(data["gs1"]) & data["lagx"].eq(1) | data["gate_id"].eq(data["gs-1"])].apply(lambda x: 1 if x >= 1 else 0)
    data.loc[(data["gate_id"].eq(3) & data["gs-1"].eq(3) & data["gs-2"].eq(10) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(3) & data["gs1"].eq(3) & data["gs-1"].eq(10) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(10) & data["gs1"].eq(3) & data["gs2"].eq(3)) |
             (data["gate_id"].eq(3) & data["gs-2"].eq(3) & data["gs-3"].eq(10) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(3) & data["gs2"].eq(3) & data["gs-1"].eq(10) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(10) & data["gs1"].eq(3) & data["gs3"].eq(3)) |
             (data["gate_id"].eq(3) & data["gs-1"].eq(3) & data["gs-3"].eq(10) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(3) & data["gs1"].eq(3) & data["gs-2"].eq(10) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(10) & data["gs2"].eq(3) & data["gs3"].eq(3)) |
             (data["gate_id"].eq(3) & data["gs-3"].eq(3) & data["gs-4"].eq(10) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(3) & data["gs3"].eq(3) & data["gs-1"].eq(10) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(10) & data["gs1"].eq(3) & data["gs4"].eq(3)) |
             (data["gate_id"].eq(3) & data["gs-2"].eq(3) & data["gs-4"].eq(10) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(3) & data["gs2"].eq(3) & data["gs-2"].eq(10) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(10) & data["gs2"].eq(3) & data["gs4"].eq(3)) |
             (data["gate_id"].eq(3) & data["gs-1"].eq(3) & data["gs-4"].eq(10) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(3) & data["gs1"].eq(3) & data["gs-3"].eq(10) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(10) & data["gs3"].eq(3) & data["gs4"].eq(3)), "xxy"] = 1
    # data["3310"] = data["gate_id"][(data["gate_id"].eq(3) & data["gs-1"].eq(3) & data["gs-2"].eq(10)) |
    #                                (data["gate_id"].eq(3) & data["gs1"].eq(3) & data["gs-1"].eq(10)) |
    #                                (data["gate_id"].eq(10) & data["gs1"].eq(3) & data["gs2"].eq(3)) |
    #                                (data["gate_id"].eq(10) & data["gs1"].eq(3) & data["gs2"].eq(3))].apply(lambda x: 1 if x >= 1 else 0)
    data.loc[(data["gate_id"].eq(11) & data["gs-1"].eq(4) & data["gs-2"].eq(4)) |
             (data["gate_id"].eq(4) & data["gs1"].eq(11) & data["gs-1"].eq(4) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(4) & data["gs1"].eq(4) & data["gs2"].eq(11) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(11) & data["gs-2"].eq(4) & data["gs-3"].eq(4)) |
             (data["gate_id"].eq(4) & data["gs2"].eq(11) & data["gs-1"].eq(4) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(4) & data["gs1"].eq(4) & data["gs3"].eq(11) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(11) & data["gs-1"].eq(4) & data["gs-3"].eq(4)) |
             (data["gate_id"].eq(4) & data["gs1"].eq(11) & data["gs-2"].eq(4) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(4) & data["gs2"].eq(4) & data["gs3"].eq(11) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(11) & data["gs-3"].eq(4) & data["gs-4"].eq(4)) |
             (data["gate_id"].eq(4) & data["gs3"].eq(11) & data["gs-1"].eq(4) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(4) & data["gs1"].eq(4) & data["gs4"].eq(11) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(11) & data["gs-2"].eq(4) & data["gs-4"].eq(4)) |
             (data["gate_id"].eq(4) & data["gs2"].eq(11) & data["gs-2"].eq(4) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(4) & data["gs2"].eq(4) & data["gs4"].eq(11) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(11) & data["gs-1"].eq(4) & data["gs-4"].eq(4)) |
             (data["gate_id"].eq(4) & data["gs1"].eq(11) & data["gs-3"].eq(4) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(4) & data["gs3"].eq(4) & data["gs4"].eq(11) & data["ls1"].eq(1)), "1144"] = 1
    # data["1144"] = data["gate_id"][(data["gate_id"].eq(11) & data["gs-1"].eq(4) & data["gs-2"].eq(4)) |
    #                                (data["gate_id"].eq(4) & data["gs1"].eq(11) & data["gs-1"].eq(4)) |
    #                                (data["gate_id"].eq(4) & data["gs1"].eq(4) & data["gs2"].eq(11))].apply(lambda x: 1 if x >= 1 else 0)
    data["73310"] = data["gate_id"][(data["gate_id"].eq(7) & data["gs-1"].eq(3) & data["gs-2"].eq(3) & data["gs-3"].eq(10)) |
                                   (data["gate_id"].eq(3) & data["gs1"].eq(7) & data["gs-1"].eq(3) & data["gs-2"].eq(10)) |
                                   (data["gate_id"].eq(3) & data["gs1"].eq(3) & data["gs2"].eq(7) & data["gs-1"].eq(10)) |
                                   (data["gate_id"].eq(10) & data["gs1"].eq(3) & data["gs2"].eq(3) & data["gs3"].eq(7))].apply(lambda x: 1 if x >= 1 else 0)
    data["6row"] = data["gate_id"][(data["gate_id"].eq(7) & data["gs-1"].eq(9) & data["gs-2"].eq(9) & data["gs-3"].eq(5) & data["gs-4"].eq(5) & data["gs-5"].eq(10)) |
                                   (data["gate_id"].eq(9) & data["gs-1"].eq(9) & data["gs-2"].eq(5) & data["gs-3"].eq(5) & data["gs-4"].eq(10) & data["gs1"].eq(7)) |
                                   (data["gate_id"].eq(9) & data["gs-1"].eq(5) & data["gs-2"].eq(5) & data["gs-3"].eq(10) & data["gs1"].eq(9) & data["gs2"].eq(7)) |
                                   (data["gate_id"].eq(5) & data["gs-1"].eq(5) & data["gs-2"].eq(10) & data["gs1"].eq(9) & data["gs2"].eq(9) & data["gs3"].eq(7)) |
                                   (data["gate_id"].eq(5) & data["gs-1"].eq(10) & data["gs1"].eq(5) & data["gs2"].eq(9) & data["gs3"].eq(9) & data["gs4"].eq(7)) |
                                    (data["gate_id"].eq(10) & data["gs1"].eq(5) & data["gs2"].eq(5) & data["gs3"].eq(9) & data["gs4"].eq(9) & data["gs5"].eq(7))]\
                                    .apply(lambda x: 1 if x >= 1 else 0)
    data["5row"] = data["gate_id"][(data["gate_id"].eq(9) & data["gs-1"].eq(9) & data["gs-2"].eq(5) & data["gs-3"].eq(5) & data["gs-4"].eq(10)) |
                                   (data["gate_id"].eq(9) & data["gs1"].eq(9) & data["gs-1"].eq(5) & data["gs-2"].eq(5) & data["gs-3"].eq(10)) |
                                   (data["gate_id"].eq(5) & data["gs1"].eq(9) & data["gs2"].eq(9) & data["gs-1"].eq(5) & data["gs-2"].eq(10)) |
                                   (data["gate_id"].eq(5) & data["gs1"].eq(5) & data["gs2"].eq(9) & data["gs3"].eq(9) & data["gs-1"].eq(10)) |
                                   (data["gate_id"].eq(10) & data["gs1"].eq(5) & data["gs2"].eq(5) & data["gs3"].eq(9) & data["gs4"].eq(9))]\
                                    .apply(lambda x: 1 if x >= 1 else 0)
    data = data.replace(np.nan, 0)
    data["beep_count"] = data.groupby("date").gate_id.transform("count")
    data = data.astype({"row_id": "int32", "gate_id": "int8", "1day": "int8", "last_day": "int8",
                        "DofW": "int8", "DofY1": "int16", "month": "int8", "day": "int8", "is_weekend": "int8",
                        "hour": "int8", "m1": "int8", "m2": "int8", "m3": "int8", "m4": "int8", "d1": "int8",
                        "d2": "int8", "d3": "int8", "d4": "int8", "e1": "int8", "e2": "int8", "e3": "int8",
                        "e4": "int8", "e5": "int8", "e6": "int8", "min": "int8", "sec": "int8", "lag": "int8",
                        "lag1": "int8", "lag2": "int8", "lag3": "int8", "lag4": "int8", "lag5": "int8", "lag6": "int8",
                        "lag7": "int8", "lag8": "int8", "lag9": "int8", "lag0": "int8", "lagx": "int8", "gs1": "int8",
                        "gs2": "int8", "gs3": "int8", "gs4": "int8", "gs5": "int8", "gs-1": "int8", "gs-2": "int8",
                        "gs-3": "int8", "gs-4": "int8", "gs-5": "int8", "double": "int8", "double2": "int8",
                        "double3": "int8", "double4": "int8", "double5": "int8", "double6": "int8", "double7": "int8",
                        "double8": "int8", '6row': 'int8', '5row': 'int8', "beep_count": "int16",
                        "1144": "int8", "73310": "int8", "l-1": "int8", "ls1": "int8", "xxy": "int8"})
    # data = data.astype({"row_id": "int32", "gate_id": "category", "1day": "category", "last_day": "category",
    #                     "DofW": "category", "DofY1": "int16", "month": "category", "day": "category", "is_weekend": "category",
    #                     "hour": "category", "m1": "category", "m2": "category", "m3": "category", "m4": "category", "d1": "category",
    #                     "d2": "category", "d3": "category", "d4": "category", "e1": "category", "e2": "category", "e3": "category",
    #                     "e4": "category", "e5": "category", "e6": "category", "min": "category", "sec": "int8", "lag": "category",
    #                     "lag1": "category", "lag2": "category", "lag3": "category", "lag4": "category", "lag5": "category", "lag6": "category",
    #                     "lag7": "category", "lag8": "category", "lag9": "category", "lag0": "int8", "lagx": "category", "gs1": "int8",
    #                     "gs2": "int8", "gs3": "int8", "gs4": "int8", "gs5": "int8", "gs-1": "int8", "gs-2": "int8",
    #                     "gs-3": "int8", "gs-4": "int8", "gs-5": "int8", "double": "category", "double2": "category",
    #                     "double3": "category", "double4": "category", "double5": "category", "double6": "category", "double7": "category",
    #                     "double8": "category", '6row': 'category', '5row': 'category', "beep_count": "int16"})
    return data

In [55]:
def preprocess(data):
    data["date"] = data["timestamp"].apply(date)
    data["time"] = data["timestamp"].apply(time)
    data["timestamp"] = pd.to_datetime(data["timestamp"])
    data["1day"] = data["timestamp"].dt.is_month_start.apply(lambda x: 1 if x == True else 0) # 1й день месяца
    data["last_day"] = data["timestamp"].dt.is_month_end.apply(lambda x: 1 if x == True else 0) # Последний день месяца
    data["DofW"] = data["timestamp"].dt.dayofweek # День недели от 0 до 6
    data["DofY1"] = (data["timestamp"].dt.dayofyear % 4).apply(lambda x: 1 if x in (1, 2) else 0) # Попытка вывести метку "график 2 через 2" пробовал иные вариации - работают хуже
    # data["DofY2"] = (data["timestamp"].dt.dayofyear % 6).apply(lambda x: 1 if x in (0, 1, 2) else 0) # "3 через 3" не прокатило
    data["month"] = data["timestamp"].dt.month
    data["day"] = data["timestamp"].dt.day
    data["is_weekend"] = data["timestamp"].apply(lambda x: 1 if x.date().weekday() in (5, 6) else 0) # Метка выходного дня
    data.loc[data["month"].eq(1) & data["day"].eq(16), "is_weekend"] = 1  #16.01 16 Января по количеству записей - праздник
    data.loc[data["month"].eq(2) & data["day"].ge(4), "is_weekend"] = 1  #after 02.03 - всё что после 3 февраля также выглядело в тестовой выборке как праздничные дни
    data.loc[data["month"].eq(9) & data["day"].eq(12), "is_weekend"] = 1  # 12.09
    data.loc[data["month"].eq(11) & data["day"].eq(11), "is_weekend"] = 1  #11.11
    data.loc[data["month"].eq(11) & data["day"].eq(16), "is_weekend"] = 1  #16.11
    data["hour"] = data["timestamp"].dt.hour
    data["m1"] = data["hour"].apply(lambda x: 1 if x == 7 else 0) # отдельно вывести некоторые часы позволило слегка повысить точность
    data["m2"] = data["hour"].apply(lambda x: 1 if x == 8 else 0)
    data["m3"] = data["hour"].apply(lambda x: 1 if x == 9 else 0)
    data["m4"] = data["hour"].apply(lambda x: 1 if x == 10 else 0)
    data["d1"] = data["hour"].apply(lambda x: 1 if x == 11 else 0)
    data["d2"] = data["hour"].apply(lambda x: 1 if x == 12 else 0)
    data["d3"] = data["hour"].apply(lambda x: 1 if x == 14 else 0)
    data["d4"] = data["hour"].apply(lambda x: 1 if x == 15 else 0)
    data["e1"] = data["hour"].apply(lambda x: 1 if x == 18 else 0)
    data["e2"] = data["hour"].apply(lambda x: 1 if x == 19 else 0)
    data["e3"] = data["hour"].apply(lambda x: 1 if x == 20 else 0)
    data["e4"] = data["hour"].apply(lambda x: 1 if x == 21 else 0)
    data["e5"] = data["hour"].apply(lambda x: 1 if x == 22 else 0)
    data["e6"] = data["hour"].apply(lambda x: 1 if x == 23 else 0)
    data["min"] = data["timestamp"].dt.minute
    data["sec"] = data["timestamp"].dt.second
    data["shift"] = data["timestamp"].shift(1) # Далее разположен блок фичей по разнице во времени срабатываний (разница в секундах)
    data["shift"] = pd.to_datetime(data["shift"])
    data["lag"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s") # Сначала подготовительный блок
    data["l3s"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s") # Отдельно выделено 13 фичей, некоторые с перекрытием
    data["lag1"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s")
    data["lag2"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s")
    data["lag3"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s")
    data["lag4"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s")
    data["lag5"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s")
    data["lag6"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s")
    data["lag7"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s")
    data["lag8"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s")
    data["lag9"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s")
    data["lag0"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s")
    data["lagx"] = (data["timestamp"] - data["shift"]) / np.timedelta64(1, "s")
    data["gs1"] = data["gate_id"].shift(1) # Также понадобилось выделить для каждой записи gate_id со смещением на 5 строк в обе стороны
    data["gs2"] = data["gate_id"].shift(2) # Далее объясню для чего
    data["gs3"] = data["gate_id"].shift(3)
    data["gs4"] = data["gate_id"].shift(4)
    data["gs5"] = data["gate_id"].shift(5)
    data["gs-1"] = data["gate_id"].shift(-1)
    data["gs-2"] = data["gate_id"].shift(-2)
    data["gs-3"] = data["gate_id"].shift(-3)
    data["gs-4"] = data["gate_id"].shift(-4)
    data["gs-5"] = data["gate_id"].shift(-5)
    data["lag"] = data["lag"].apply(lambda x: 1 if x < 2 else 0) # Перевод фичей с разницей срабатываний в нули и единицы
    data["l3s"] = data["l3s"].apply(lambda x: 1 if x <= 3 else 0) # Отдельно выделен диапазон <=3 сек
    data["l-1"] = data["l3s"].shift(-1)
    data.loc[data["l3s"].eq(1) | data["l-1"].eq(1), "ls1"] = 1
    data["lag1"] = data["lag1"].apply(lambda x: 1 if 6 > x > 2 else 0)
    data["lag2"] = data["lag2"].apply(lambda x: 1 if 15 > x >= 6 else 0)
    data["lag3"] = data["lag3"].apply(lambda x: 1 if 22 > x >= 15 else 0)
    data["lag4"] = data["lag4"].apply(lambda x: 1 if 32 > x >= 22 else 0)
    data["lag5"] = data["lag5"].apply(lambda x: 1 if 42 > x > 32 else 0)
    data["lag6"] = data["lag6"].apply(lambda x: 1 if 58 > x >= 42 else 0)
    data["lag7"] = data["lag7"].apply(lambda x: 1 if 69 > x >= 58 else 0)
    data["lag8"] = data["lag8"].apply(lambda x: 1 if 76 > x >= 69 else 0)
    data["lag9"] = data["lag9"].apply(lambda x: 1 if 130 >= x > 127 else 0)
    data["lag0"] = data["lag0"].apply(lambda x: 0 if x > 69 else x)
    data["lagx"] = data["lagx"].apply(lambda x: 1 if x > 130 else 0)
    # Блок фичей с использованием разницы срабатываний и сравнением gate_id соседних записей
    data["double"] = data["gate_id"][data["gate_id"].eq(data["gs1"]) & data["ls1"].eq(1) | data["gate_id"].eq(data["gs-1"])].apply(lambda x: 1 if x >= 1 else 0)
    data["double2"] = data["gate_id"][data["gate_id"].eq(data["gs1"]) & data["lag"].eq(0) | data["gate_id"].eq(data["gs-1"])].apply(lambda x: 1 if x >= 1 else 0)
    data["double3"] = data["gate_id"][data["gate_id"].eq(data["gs1"]) & data["lag1"].eq(1) | data["gate_id"].eq(data["gs-1"])].apply(lambda x: 1 if x >= 1 else 0)
    data["double4"] = data["gate_id"][data["gate_id"].eq(data["gs1"]) & data["lag2"].eq(1) | data["gate_id"].eq(data["gs-1"])].apply(lambda x: 1 if x >= 1 else 0)
    data["double5"] = data["gate_id"][data["gate_id"].eq(data["gs1"]) & data["lag3"].eq(1) | data["gate_id"].eq(data["gs-1"])].apply(lambda x: 1 if x >= 1 else 0)
    data["double6"] = data["gate_id"][data["gate_id"].eq(data["gs1"]) & data["lag4"].eq(1) | data["gate_id"].eq(data["gs-1"])].apply(lambda x: 1 if x >= 1 else 0)
    data["double7"] = data["gate_id"][data["gate_id"].eq(data["gs1"]) & data["lag7"].eq(1) | data["gate_id"].eq(data["gs-1"])].apply(lambda x: 1 if x >= 1 else 0)
    data["double8"] = data["gate_id"][data["gate_id"].eq(data["gs1"]) & data["lagx"].eq(1) | data["gate_id"].eq(data["gs-1"])].apply(lambda x: 1 if x >= 1 else 0)
    # Далее интересное: при детальном рассмотрении трейн и тестовых данных было замечен паттерн последовательности
    # проходов одним юзером через гейты 3-3-10. Также в эту последовательность может вклиниться другой юзер с одним или двумя проходами
    # Данная реализация содержит часть ложных срабатываний, как это пофиксить пока не придумал.
    # То место где понадобятся смещения!
    data.loc[(data["gate_id"].eq(3) & data["gs-1"].eq(3) & data["gs-2"].eq(10) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(3) & data["gs1"].eq(3) & data["gs-1"].eq(10) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(10) & data["gs1"].eq(3) & data["gs2"].eq(3)) |
             (data["gate_id"].eq(3) & data["gs-2"].eq(3) & data["gs-3"].eq(10) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(3) & data["gs2"].eq(3) & data["gs-1"].eq(10) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(10) & data["gs1"].eq(3) & data["gs3"].eq(3)) |
             (data["gate_id"].eq(3) & data["gs-1"].eq(3) & data["gs-3"].eq(10) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(3) & data["gs1"].eq(3) & data["gs-2"].eq(10) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(10) & data["gs2"].eq(3) & data["gs3"].eq(3)) |
             (data["gate_id"].eq(3) & data["gs-3"].eq(3) & data["gs-4"].eq(10) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(3) & data["gs3"].eq(3) & data["gs-1"].eq(10) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(10) & data["gs1"].eq(3) & data["gs4"].eq(3)) |
             (data["gate_id"].eq(3) & data["gs-2"].eq(3) & data["gs-4"].eq(10) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(3) & data["gs2"].eq(3) & data["gs-2"].eq(10) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(10) & data["gs2"].eq(3) & data["gs4"].eq(3)) |
             (data["gate_id"].eq(3) & data["gs-1"].eq(3) & data["gs-4"].eq(10) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(3) & data["gs1"].eq(3) & data["gs-3"].eq(10) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(10) & data["gs3"].eq(3) & data["gs4"].eq(3)), "xxy"] = 1
    # похожий паттерн, только через гейты 11-4-4
    data.loc[(data["gate_id"].eq(11) & data["gs-1"].eq(4) & data["gs-2"].eq(4)) |
             (data["gate_id"].eq(4) & data["gs1"].eq(11) & data["gs-1"].eq(4) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(4) & data["gs1"].eq(4) & data["gs2"].eq(11) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(11) & data["gs-2"].eq(4) & data["gs-3"].eq(4)) |
             (data["gate_id"].eq(4) & data["gs2"].eq(11) & data["gs-1"].eq(4) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(4) & data["gs1"].eq(4) & data["gs3"].eq(11) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(11) & data["gs-1"].eq(4) & data["gs-3"].eq(4)) |
             (data["gate_id"].eq(4) & data["gs1"].eq(11) & data["gs-2"].eq(4) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(4) & data["gs2"].eq(4) & data["gs3"].eq(11) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(11) & data["gs-3"].eq(4) & data["gs-4"].eq(4)) |
             (data["gate_id"].eq(4) & data["gs3"].eq(11) & data["gs-1"].eq(4) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(4) & data["gs1"].eq(4) & data["gs4"].eq(11) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(11) & data["gs-2"].eq(4) & data["gs-4"].eq(4)) |
             (data["gate_id"].eq(4) & data["gs2"].eq(11) & data["gs-2"].eq(4) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(4) & data["gs2"].eq(4) & data["gs4"].eq(11) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(11) & data["gs-1"].eq(4) & data["gs-4"].eq(4)) |
             (data["gate_id"].eq(4) & data["gs1"].eq(11) & data["gs-3"].eq(4) & data["ls1"].eq(1)) |
             (data["gate_id"].eq(4) & data["gs3"].eq(4) & data["gs4"].eq(11) & data["ls1"].eq(1)), "1144"] = 1
    # начальная(не очень умная) реализация паттерна 7-3-3-10
    data["73310"] = data["gate_id"][(data["gate_id"].eq(7) & data["gs-1"].eq(3) & data["gs-2"].eq(3) & data["gs-3"].eq(10)) |
                                   (data["gate_id"].eq(3) & data["gs1"].eq(7) & data["gs-1"].eq(3) & data["gs-2"].eq(10)) |
                                   (data["gate_id"].eq(3) & data["gs1"].eq(3) & data["gs2"].eq(7) & data["gs-1"].eq(10)) |
                                   (data["gate_id"].eq(10) & data["gs1"].eq(3) & data["gs2"].eq(3) & data["gs3"].eq(7))].apply(lambda x: 1 if x >= 1 else 0)
    # паттерн 7-9-9-5-5-10
    data["6row"] = data["gate_id"][(data["gate_id"].eq(7) & data["gs-1"].eq(9) & data["gs-2"].eq(9) & data["gs-3"].eq(5) & data["gs-4"].eq(5) & data["gs-5"].eq(10)) |
                                   (data["gate_id"].eq(9) & data["gs-1"].eq(9) & data["gs-2"].eq(5) & data["gs-3"].eq(5) & data["gs-4"].eq(10) & data["gs1"].eq(7)) |
                                   (data["gate_id"].eq(9) & data["gs-1"].eq(5) & data["gs-2"].eq(5) & data["gs-3"].eq(10) & data["gs1"].eq(9) & data["gs2"].eq(7)) |
                                   (data["gate_id"].eq(5) & data["gs-1"].eq(5) & data["gs-2"].eq(10) & data["gs1"].eq(9) & data["gs2"].eq(9) & data["gs3"].eq(7)) |
                                   (data["gate_id"].eq(5) & data["gs-1"].eq(10) & data["gs1"].eq(5) & data["gs2"].eq(9) & data["gs3"].eq(9) & data["gs4"].eq(7)) |
                                    (data["gate_id"].eq(10) & data["gs1"].eq(5) & data["gs2"].eq(5) & data["gs3"].eq(9) & data["gs4"].eq(9) & data["gs5"].eq(7))]\
                                    .apply(lambda x: 1 if x >= 1 else 0)
    # паттерн 9-9-5-5-10
    data["5row"] = data["gate_id"][(data["gate_id"].eq(9) & data["gs-1"].eq(9) & data["gs-2"].eq(5) & data["gs-3"].eq(5) & data["gs-4"].eq(10)) |
                                   (data["gate_id"].eq(9) & data["gs1"].eq(9) & data["gs-1"].eq(5) & data["gs-2"].eq(5) & data["gs-3"].eq(10)) |
                                   (data["gate_id"].eq(5) & data["gs1"].eq(9) & data["gs2"].eq(9) & data["gs-1"].eq(5) & data["gs-2"].eq(10)) |
                                   (data["gate_id"].eq(5) & data["gs1"].eq(5) & data["gs2"].eq(9) & data["gs3"].eq(9) & data["gs-1"].eq(10)) |
                                   (data["gate_id"].eq(10) & data["gs1"].eq(5) & data["gs2"].eq(5) & data["gs3"].eq(9) & data["gs4"].eq(9))]\
                                    .apply(lambda x: 1 if x >= 1 else 0)
    data = data.replace(np.nan, 0) # Замена NaN из колонок со смещением и не только
    data["beep_count"] = data.groupby("date").gate_id.transform("count") # Подсчет количества срабатываний за день
    # Переводим типы данных в минимально допустимые - экономим ресурсы
    data = data.astype({"row_id": "int32", "user_id": "int8", "gate_id": "int8", "1day": "int8", "last_day": "int8",
                        "DofW": "int8", "DofY1": "int16", "month": "int8", "day": "int8", "is_weekend": "int8",
                        "hour": "int8", "m1": "int8", "m2": "int8", "m3": "int8", "m4": "int8", "d1": "int8",
                        "d2": "int8", "d3": "int8", "d4": "int8", "e1": "int8", "e2": "int8", "e3": "int8",
                        "e4": "int8", "e5": "int8", "e6": "int8", "min": "int8", "sec": "int8", "lag": "int8",
                        "lag1": "int8", "lag2": "int8", "lag3": "int8", "lag4": "int8", "lag5": "int8", "lag6": "int8",
                        "lag7": "int8", "lag8": "int8", "lag9": "int8", "lag0": "int8", "lagx": "int8", "gs1": "int8",
                        "gs2": "int8", "gs3": "int8", "gs4": "int8", "gs5": "int8", "gs-1": "int8", "gs-2": "int8",
                        "gs-3": "int8", "gs-4": "int8", "gs-5": "int8", "double": "int8", "double2": "int8",
                        "double3": "int8", "double4": "int8", "double5": "int8", "double6": "int8", "double7": "int8",
                        "double8": "int8", '6row': 'int8', '5row': 'int8', "beep_count": "int16",
                        "1144": "int8", "73310": "int8", "l-1": "int8", "ls1": "int8", "xxy": "int8"})
    # data = data.astype({"row_id": "int32", "user_id": "category", "gate_id": "category", "1day": "category", "last_day": "category",
    #                     "DofW": "category", "DofY1": "int16", "month": "category", "day": "category", "is_weekend": "category",
    #                     "hour": "category", "m1": "category", "m2": "category", "m3": "category", "m4": "category", "d1": "category",
    #                     "d2": "category", "d3": "category", "d4": "category", "e1": "category", "e2": "category", "e3": "category",
    #                     "e4": "category", "e5": "category", "e6": "category", "min": "category", "sec": "int8", "lag": "category",
    #                     "lag1": "category", "lag2": "category", "lag3": "category", "lag4": "category", "lag5": "category", "lag6": "category",
    #                     "lag7": "category", "lag8": "category", "lag9": "category", "lag0": "int8", "lagx": "category", "gs1": "int8",
    #                     "gs2": "int8", "gs3": "int8", "gs4": "int8", "gs5": "int8", "gs-1": "int8", "gs-2": "int8",
    #                     "gs-3": "int8", "gs-4": "int8", "gs-5": "int8", "double": "category", "double2": "category",
    #                     "double3": "category", "double4": "category", "double5": "category", "double6": "category", "double7": "category",
    #                     "double8": "category", '6row': 'category', '5row': 'category', "beep_count": "int16"})

    return data

In [56]:
import pandas as pd
df = pd.read_csv('/kaggle/working/train.csv', parse_dates=['timestamp'])
df = clear_usrer_df(df,df['user_id']) # работаем с топ 35 user_id по популрности

In [57]:
labels = df['user_id']
df_train = preprocess(df.astype({'timestamp':str}))
df_train['time_hour'] = df.timestamp.apply(lambda x: x.day_of_week * 24 + x.hour)
df_train['all_time'] = df.timestamp.apply(lambda x:x.hour*60 +x.minute + x.second / 60)

In [58]:
filln = df_train['all_time'][0]
df_train['time_shift1'] = df_train.shift(1)['all_time'].fillna(filln)
df_train['time_shift-1'] = df_train.shift(-1)['all_time'].fillna(filln)

In [59]:
# промежуток между следующем и нынешнем временем
df_train['time_step1'] = np.abs(df_train['time_shift1'] - df_train['all_time'])
df_train['time_step-1'] = np.abs(df_train['all_time'] - df_train['time_shift-1'])

In [60]:
# Label encode
labels_dict = pd.DataFrame(list(range(labels.value_counts().shape[0])),labels.value_counts().index).to_dict()[0]
labels = list(map(lambda x:labels_dict[x],labels))

In [61]:
trained_columns = ["gate_id", "DofW", "is_weekend", "1day", "last_day", "hour", "min", "lag", "lag1", "lag2", "lag3", "lag4", "lag5", "lag6", "lag7", "lag8", "lag9", "lag0",
                   "lagx","m1","m2","m3","m4","d1","d2","d3","d4","e1","e2","e3","e4","e5","e6", "6row", "5row", "beep_count", "1144", "73310", "xxy",
                   "double", "double2", "double3", "double4", "double5", "double6", "double7", "double8",'all_time','DofY1','gs-1','gs1','time_hour',
                  ]

In [80]:
params = {
    'loss_function':'Logloss',
    'task_type': 'CPU',
    'random_seed':42,
    'iterations': 420,
    'eval_metric': 'Accuracy',
    'max_depth':4,
    }

In [63]:
# если временной промежуток между проходами меньше 90 от мы его выводим, иначе -1
df_train['cat_step'] = (df_train['time_step1'] * 60).astype(int)
df_train['cat_step'] = df_train['cat_step'].apply(lambda x: x if x<= 90 else -1)

In [64]:
# если предыдущий турникет равен следующему 
df_train['super_count'] = 1 
df_train['super_count'] = df_train[df_train['gs1'] == df_train['gs-1']]['super_count']
df_train['super_count'] =  df_train['super_count'].fillna(0).astype(int)

In [65]:
# функця создания ансамбля catboost, где каждая модель предсказывает свой user_id 
from tqdm import tqdm
def create_ensembles(X,y,params,label_dict,cat_feat=['gate_id','gs1','gs-1','1144','73310','gs2','gs-2','gs3','gs-3','min','gs4','gs-4','gs5','gs-5']):
    model_list = []
    for label in tqdm(label_dict):
        y_ft = y.apply(lambda x: 1 if x == label else 0)
        model = CatBoostClassifier(**params)
        model.fit(X,y_ft,cat_features=cat_feat,verbose=False)
        model_list += [model]
    return model_list

In [101]:
# функция для предсказания ансамбля
def ensemble_predict(model_list,X):
    preds = np.stack([model.predict_proba(X)[:,1] for model in model_list])
    preds[0] *= 0.9 # В тестовых данных много выходных, а на выходных этот user_id менее активен
    preds[2] *= 1.7 # В тестовых данных много выходных, а на выходных этот user_id более активен
    preds[3] *= 1.15
    return np.argmax(preds,axis=0)

In [67]:
trained_columns += ['gs2','gs-2']

In [68]:
trained_columns += ['gs3','gs-3','gs4','gs-4','gs5','gs-5']

In [69]:
trained_columns += ['time_step1','time_step-1','cat_step','super_count']

In [70]:
# берём следующий и предыдущий временной шаг (time_step)
filln = df_train['time_step1'][0]
df_train['next_step1'] = df_train.shift(1)['time_step1'].fillna(filln)
df_train['last_step1'] = df_train.shift(-1)['time_step1'].fillna(filln)

filln = df_train['time_step-1'][0]
df_train['next_step-1'] = df_train.shift(1)['time_step-1'].fillna(filln)
df_train['last_step-1'] = df_train.shift(-1)['time_step-1'].fillna(filln)

In [71]:
trained_columns += ['next_step1','last_step1','next_step-1','last_step-1']

In [72]:
filln = df_train['time_step1'][0]
df_train['2next_step1'] = df_train.shift(2)['time_step1'].fillna(filln)
df_train['2last_step1'] = df_train.shift(-2)['time_step1'].fillna(filln)

filln = df_train['time_step-1'][0]
df_train['2next_step-1'] = df_train.shift(2)['time_step-1'].fillna(filln)
df_train['2last_step-1'] = df_train.shift(-2)['time_step-1'].fillna(filln)

In [73]:
trained_columns += ['2next_step1','2last_step1','2next_step-1','2last_step-1']

In [74]:
# разница между временными шагами
df_train['delta_1'] = np.abs(df_train['next_step1']-df_train['time_step1'])
df_train['delta_-1'] = np.abs(df_train['next_step-1']-df_train['time_step-1'])
trained_columns += ['delta_1','delta_-1']

In [75]:
df_train['2delta_1'] = np.abs(df_train['2next_step1']-df_train['next_step1'])
df_train['2delta_-1'] = np.abs(df_train['2next_step-1']-df_train['next_step-1'])
trained_columns += ['2delta_1','2delta_-1']

In [76]:
filln = df_train['cat_step'][0]
df_train['1shift_cat_step'] = df_train.shift(1).cat_step.fillna(filln)
df_train['-1shift_cat_step'] = df_train.shift(-1).cat_step.fillna(filln)

In [77]:
#model_list = create_ensembles(X_train,pd.Series(y_train),params,labels_dict.values())

In [78]:
#from sklearn.metrics import accuracy_score
#y_p = ensemble_predict(model_list,X_test)
#accuracy_score(y_test,y_p)

In [81]:
all_ensemble = create_ensembles(df_train[trained_columns],pd.Series(labels),params,labels_dict.values())

100%|██████████| 35/35 [12:20<00:00, 21.14s/it]


In [33]:
!wget https://storage.yandexcloud.net/ds-ods/files/materials/471ec182/test.csv 
!wget https://storage.yandexcloud.net/ds-ods/files/materials/9a9fc693/sample_submission.sv

--2023-05-01 19:18:15--  https://storage.yandexcloud.net/ds-ods/files/materials/471ec182/test.csv
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 201790 (197K) [text/csv]
Saving to: ‘test.csv’

test.csv            100%[===================>] 197.06K   363KB/s    in 0.5s    

2023-05-01 19:18:16 (363 KB/s) - ‘test.csv’ saved [201790/201790]

--2023-05-01 19:18:17--  https://storage.yandexcloud.net/ds-ods/files/materials/9a9fc693/sample_submission.sv
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 63111 (62K) [application/octet-stream]
Saving to: ‘sample_submission.sv’

sample_submission.s 100%[=

In [82]:
df_test = pd.read_csv("/kaggle/working/test.csv", parse_dates=['timestamp'])
test_data = preprocess_test(df_test.astype({'timestamp':str}))
test_data['all_time'] = df_test.timestamp.apply(lambda x:x.hour*60 +x.minute + x.second / 60)
test_data['time_hour'] = df_test.timestamp.apply(lambda x: x.day_of_week * 24 + x.hour)

In [83]:
filln = test_data['all_time'][0]
test_data['time_shift1'] = test_data.shift(1)['all_time'].fillna(filln)
test_data['time_shift-1'] = test_data.shift(-1)['all_time'].fillna(filln)

In [84]:
test_data['time_step1'] = np.abs(test_data['time_shift1'] - test_data['all_time'])
test_data['time_step-1'] = np.abs(test_data['all_time'] - test_data['time_shift-1'])

In [85]:
filln = test_data['time_step1'][0]
test_data['next_step1'] = test_data.shift(1)['time_step1'].fillna(filln)
test_data['last_step1'] = test_data.shift(-1)['time_step1'].fillna(filln)

filln = test_data['time_step-1'][0]
test_data['next_step-1'] = test_data.shift(1)['time_step-1'].fillna(filln)
test_data['last_step-1'] = test_data.shift(-1)['time_step-1'].fillna(filln)

In [86]:
filln = test_data['time_step1'][0]
test_data['2next_step1'] = test_data.shift(2)['time_step1'].fillna(filln)
test_data['2last_step1'] = test_data.shift(-2)['time_step1'].fillna(filln)

filln = test_data['time_step-1'][0]
test_data['2next_step-1'] = test_data.shift(2)['time_step-1'].fillna(filln)
test_data['2last_step-1'] = test_data.shift(-2)['time_step-1'].fillna(filln)

In [87]:
test_data['delta_1'] = np.abs(test_data['next_step1']-test_data['time_step1'])
test_data['delta_-1'] = np.abs(test_data['next_step-1']-test_data['time_step-1'])

In [88]:
test_data['2delta_1'] = np.abs(test_data['2next_step1']-test_data['next_step1'])
test_data['2delta_-1'] = np.abs(test_data['2next_step-1']-test_data['next_step-1'])

In [89]:
test_data['cat_step'] = (test_data['time_step1'] * 60).astype(int)
test_data['cat_step'] = test_data['cat_step'].apply(lambda x: x if x<= 90 else -1)

In [90]:
test_data['super_count'] = 1
test_data['super_count'] = test_data[test_data['gs1'] == test_data['gs-1']]['super_count']
test_data['super_count'] =  test_data['super_count'].fillna(0).astype(int)

In [91]:
filln = test_data['cat_step'][0]
test_data['1shift_cat_step'] = test_data.shift(1).cat_step.fillna(filln)
test_data['-1shift_cat_step'] = test_data.shift(-1).cat_step.fillna(filln)

In [92]:
test_data['1shift_1144'] = test_data.shift(1)['1144'].fillna(0)
test_data['-1shift_1144'] = test_data.shift(-1)['1144'].fillna(0)

In [93]:
test_data['time_mean'] = test_data.groupby('time').transform('count').gate_id

In [102]:
targets = ensemble_predict(all_ensemble,test_data[trained_columns])
targets = list(map(lambda x:list(labels_dict.keys())[x],targets.reshape(-1)))

In [103]:
sample_submit = pd.read_csv('/kaggle/working/sample_submission.sv',)
sample_submit['target'] = targets

In [104]:
pd.Series(sample_submit.target).value_counts()

6     852
12    687
55    640
15    415
32    395
37    348
33    344
19    336
1     326
49    252
54    252
3     247
17    239
11    234
14    218
0     216
35    180
39    153
29    124
50    119
47    113
18     85
9      77
23     59
24     53
53     37
26     36
46     28
48     27
42     16
27     11
25      3
57      2
34      1
Name: target, dtype: int64

In [105]:
sample_submit.to_csv('submit_x.csv',index=False)

In [100]:
sample_submit

,row_id,target
0,37518,1
1,37519,1
2,37520,15
3,37521,15
4,37522,15
...,...,...
7120,44638,49
7121,44639,49
7122,44640,49
7123,44641,1


public - 0,2877596856

private - 0.2845916363